In [ ]:
# Some imports we will need for this notebook
import Augmentor
import numpy as np
from PIL import Image
import glob
from natsort import natsorted
import os
import random
import requests
import zipfile
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Reading and sorting the image paths from the directories
gt_binary_image = natsorted(glob.glob("/aimldl-dat/data-gaze/AIML_Aids/lnd-251119_114944/training/gt_binary_image/*.png"))
gt_image = natsorted(glob.glob("/aimldl-dat/data-gaze/AIML_Aids/lnd-251119_114944/training/gt_image/*.png"))
gt_instance_image = natsorted(glob.glob("/aimldl-dat/data-gaze/AIML_Aids/lnd-251119_114944/training/gt_instance_image/*.png"))

In [ ]:
for i in range(0, len(gt_binary_image)):
    print("%s: Binary: %s | Original: %s | Instance: %s" % 
          (i+1, os.path.basename(gt_binary_image[i]),
           os.path.basename(gt_image[i]),
           os.path.basename(gt_instance_image[i])))

In [ ]:
collated_images_and_masks = list(zip(gt_binary_image, 
                                     gt_image, 
                                     gt_instance_image))
collated_images_and_masks

In [ ]:
images = [[np.asarray(Image.open(y)) for y in x] for x in collated_images_and_masks]

In [ ]:
# r_index = random.randint(0, len(images)-1)
for r_index in range(len(images)):
    f, axarr = plt.subplots(1, 3, figsize=(20,15))
    axarr[0].imshow(images[r_index][0])
    axarr[1].imshow(images[r_index][1], cmap="gray")
    axarr[2].imshow(images[r_index][2], cmap="gray");

In [ ]:
p = Augmentor.DataPipeline(images)

In [ ]:
# p.rotate(1, max_left_rotation=5, max_right_rotation=5)
p.rotate_without_crop(1, max_left_rotation=25, max_right_rotation=25, expand=True)


## Dont use for now
# p.rotate_random_90(1)
# p.zoom_random(1, percentage_area=0.5)

In [ ]:
sample_size = 10
augmented_images = p.sample(sample_size)

In [ ]:
for r_index in range(sample_size):
    f, axarr = plt.subplots(1, 3, figsize=(20,15))
    axarr[0].imshow(augmented_images[r_index][0])
    axarr[1].imshow(augmented_images[r_index][1], cmap="gray")
    axarr[2].imshow(augmented_images[r_index][2], cmap="gray");

In [ ]:
out_path = "/aimldl-dat/logs/testing/lanenet-aug/output"

gt_instance_image_path = os.path.join(out_path,"gt_instance_image")
gt_image_path = os.path.join(out_path,"gt_image")
gt_binary_image_path = os.path.join(out_path,"gt_binary_image")

os.mkdir(gt_instance_image_path)
os.mkdir(gt_image_path)
os.mkdir(gt_binary_image_path)

In [ ]:
img_paths = [gt_binary_image_path, gt_image_path, gt_instance_image_path]
for i in range(len(augmented_images)):
    for j,path in enumerate(img_paths):
        im = Image.fromarray(augmented_images[i][j])
        imgname = str(i)+".jpeg"
        im.save(os.path.join(path, imgname))